In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')







In [ ]:
df.head()

SOME EDA ON OUR DATASET

check for if there are empty data or not!

In [ ]:
df.isnull().any()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns
sns.heatmap(df.isnull())

so we confirmed , we have no missing data

since our target columns is 'Selling_Price' , we would se how it is affected by other variables

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.heatmap(df.corr())

In [ ]:
plt.scatter(df['Present_Price'],df['Selling_Price'])
plt.xlabel('Present Price')
plt.ylabel('Selling Price')

as we know this is positively co-related data so as the 'present price' is increased thus 'selling price' would also increase 

In [ ]:
plt.scatter(df['Kms_Driven'],df['Selling_Price'])

# NUMERICAL FEATURES

In [ ]:
#list of numerical values
numerical_features = [feature for feature in df.columns if df[feature].dtypes != 'O']

print('number of numerical variables', len(numerical_features))
print('total features ',len(df.columns))

df[numerical_features].head()

YEAR FEATURE

In [ ]:
df.groupby('Year')['Selling_Price'].median().plot()
plt.xlabel('Year')
plt.ylabel('Selling Price')

but since currently we are in year 2020 so we would just see the year difference and see how prices are varying with the year difference

In [ ]:
df['year_diff'] = 2020 - df['Year']
df = df.drop('Year',axis=1)
df.head()

visualisng how the years are varying with selling price

In [ ]:
plt.scatter(df['year_diff'],df['Selling_Price'])
plt.xlabel('Year Difference')
plt.ylabel('Selling Price')

# DISCRETE AND CONTINOUS NUMERICAL FEATURES

we can see that dicrete feature in numerical feature is only Owner

In [ ]:
data = df.copy()
data.groupby(df['Owner'])['Selling_Price'].median().plot.bar()
plt.xlabel('No of owners previously')
plt.ylabel('Selling price')

now we would see continous features 

In [ ]:
continous_feature = [df['Kms_Driven'],df['year_diff'],df['Present_Price']]

In [ ]:
df['Kms_Driven'].hist(bins=25)

In [ ]:
df['year_diff'].hist(bins=25)

In [ ]:
df['Present_Price'].hist(bins=25)

so most of our continous data is not normal , to practice regression we should have normally distributed data!, we would try to covert our skewed data into normal data using log conversions !

In [ ]:

df['Kms_Driven'] = np.log(df['Kms_Driven'])
df['Present_Price'] = np.log(df['Present_Price'])

In [ ]:
df['Kms_Driven'].hist(bins=25)

In [ ]:
df['Present_Price'].hist(bins=25)

so we have roughly converted our datasets into normally distributes (roughly)

CATEGORICAL FEATURES

In [ ]:
categorical_features = [feature for feature in df.columns if df[feature].dtypes=='O']

In [ ]:
df[categorical_features].head()

In [ ]:
df['Car_Name'].value_counts()


In [ ]:
df['Fuel_Type'].value_counts()

In [ ]:
df['Seller_Type'].value_counts()

In [ ]:
df['Transmission'].value_counts()

In [ ]:
for feature in categorical_features:
    data = df.copy()
    data.groupby(feature)['Selling_Price'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('Sale Price')
    plt.show()

SO this is enough EDA for our dataset , now we would do some feature engineering

# FEATURE ENGINNERING

first thing that we dont need here is name of the car 

In [ ]:
df = df.drop('Car_Name',axis = 1)

In [ ]:
df.head()

In [ ]:
final_dataset = df

In [ ]:
final_dataset = pd.get_dummies(final_dataset,drop_first=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.columns

In [ ]:
x = final_dataset[['Present_Price', 'Kms_Driven', 'Owner', 'year_diff',
       'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'Seller_Type_Individual',
       'Transmission_Manual']]
y = final_dataset['Selling_Price']

In [ ]:
x.head()

In [ ]:
y.head()

MODEL

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.25,random_state = 0)

LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)
print(linear_model.score(x_test,y_test))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(x_train,y_train)
print(tree.score(x_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest =  RandomForestRegressor()
forest.fit(x_train,y_train)
print(forest.score(x_test,y_test))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
grad_boost = GradientBoostingRegressor()
grad_boost.fit(x_train,y_train)
print(grad_boost.score(x_test,y_test))

In [ ]:
import xgboost
from xgboost import XGBRegressor
x_model = XGBRegressor()
x_model.fit(x_train,y_train)
print(x_model.score(x_test,y_test))